<h3>1. Загружаем и исследуем данные</h3>

In [1]:
# Импортируем библиотеки
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import seaborn as sns
from  matplotlib import pyplot as plt
from urllib.parse import urlencode
import requests
import json

In [2]:
# Функция для загрузки файлов с Яндекс диска
def yandex_load(public_key):
    # Получаем загрузочную ссылку
    yandex_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    final_url = yandex_url + urlencode(dict(public_key=public_key))
    # запрос ссылки на скачивание
    response = requests.get(final_url)
    # 'парсинг' ссылки на скачивание
    url = response.json()['href'] 
    return url

In [3]:
# Ссылки на файлы в Яндекс диске
yandex_groups = 'https://disk.yandex.ru/d/UhyYx41rTt3clQ' 
yandex_groups_add = 'https://disk.yandex.ru/d/5Kxrz02m3IBUwQ'
yandex_active_studs = 'https://disk.yandex.ru/d/Tbs44Bm6H_FwFQ'
yandex_checks = 'https://disk.yandex.ru/d/pH1q-VqcxXjsVA'

In [4]:
# Загружаем таблицу с информацией о принадлежности пользователя к контрольной или экспериментальной группе (разделитель ;)
groups = pd.read_csv(yandex_load(yandex_groups), sep=';')
groups.head()

,id,grp
0,1489,B
1,1627,A
2,1768,B
3,1783,B
4,1794,A


In [5]:
# Загружаем дополнительную таблицу с группами
groups_add = pd.read_csv(yandex_load(yandex_groups_add))
groups_add.head()

,id,grp
0,5694584,B
1,5694830,B
2,5695057,B
3,5698872,B
4,5699067,B


In [6]:
# Загружаем таблицу с активными пользователями в дни проведения эксперимента
active_studs = pd.read_csv(yandex_load(yandex_active_studs))
active_studs.head()

,student_id
0,581585
1,5723133
2,3276743
3,4238589
4,4475369


In [7]:
# Загружаем таблицу с оплатами пользователей в дни проведения эксперимента (разделитель ;)
checks= pd.read_csv(yandex_load(yandex_checks), sep=';')
checks.head()

,student_id,rev
0,1627,990.0
1,3185,690.0
2,25973,690.0
3,26280,690.0
4,100300,990.0


In [8]:
# Объединим основные данные по группам с дополнительными
groups_full = pd.concat([groups, groups_add])
groups_full.head()

,id,grp
0,1489,B
1,1627,A
2,1768,B
3,1783,B
4,1794,A


In [9]:
# Проведем предварительное исследование данных, начнем с таблицы с группами
groups_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74576 entries, 0 to 91
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      74576 non-null  int64 
 1   grp     74576 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.7+ MB


In [10]:
# Пропущенных значений нет, всего 74576 строк
# Проверим количество уникальных пользователей
groups_full.id.nunique()

74576

In [11]:
# Совпадает с количеством строк, значит, дубликатов нет
groups_full.dtypes

id      int64
grp    object
dtype: object

In [12]:
# Типы данных корректны
# Посмотрим на таблицу с активными пользователями
active_studs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8341 entries, 0 to 8340
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   student_id  8341 non-null   int64
dtypes: int64(1)
memory usage: 65.3 KB


In [13]:
# Количество строк 8341, есть ли дубликаты?
active_studs.student_id.nunique()

8341

In [14]:
# Дубликатов нет
active_studs.dtypes

student_id    int64
dtype: object

In [15]:
# Типы данных в порядке
# Рассмотрим таблицу с оплатами
checks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541 entries, 0 to 540
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   student_id  541 non-null    int64  
 1   rev         541 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 8.6 KB


In [16]:
# 541 строка без пропусков, посмотрим сколько уникальных пользователей
checks.student_id.nunique()

541

In [17]:
# Дубликатов нет. Типы данных в порядке, тип данных для id пользователя во всех таблицах одинаковые 

In [22]:
# Объединим таблицу с группами и таблицу с пользователями и переименуем колонку student_id в active:
groups_active_studs = pd.merge(groups_full, active_studs, how='left', left_on='id', right_on='student_id').rename(columns={'student_id': 'active'})
groups_active_studs.head()

,id,grp,active
0,1489,B,NaN
1,1627,A,1627.0
2,1768,B,NaN
3,1783,B,NaN
4,1794,A,NaN


In [23]:
# Заменим NaN в колонке active на 0:
groups_active_studs = groups_active_studs.fillna(0)
groups_active_studs

,id,grp,active
0,1489,B,0.0
1,1627,A,1627.0
2,1768,B,0.0
3,1783,B,0.0
4,1794,A,0.0
...,...,...,...
74571,200247820,B,0.0
74572,201032527,B,0.0
74573,201067612,B,0.0
74574,201067653,B,0.0


In [32]:
# Объединим полученную таблицу с таблицей с оплатами:
data_full = pd.merge(groups_active_studs, checks, how='left', left_on = 'id' , right_on='student_id').fillna(0)
data_full.head()

,id,grp,active,student_id,rev
0,1489,B,0.0,0.0,0.0
1,1627,A,1627.0,1627.0,990.0
2,1768,B,0.0,0.0,0.0
3,1783,B,0.0,0.0,0.0
4,1794,A,0.0,0.0,0.0


In [33]:
data_full.shape

(74576, 5)

In [34]:
data_full.id.nunique()

74576

In [35]:
# Количество строк в получившейся таблице совпадает с количеством уникалных id, т.е. у каждого пользователя не более 1 покупки
# Проверим, есть ли совершившие оплату, но не активные
data_full.query('active == 0 and rev > 0').id.count()

149

In [39]:
# 149 человек совершили оплату, но не попали в активные. Т.е. они не заходили на сайт и не видели новую механику оплаты,
# значит они не представляют интереса для эксперимента. Будем работать только с активными.
data = data_full.query('active > 0')
data.head()

,id,grp,active,student_id,rev
1,1627,A,1627.0,1627.0,990.0
10,2085,B,2085.0,0.0,0.0
12,2215,B,2215.0,0.0,0.0
43,3391,A,3391.0,0.0,0.0
45,3401,B,3401.0,0.0,0.0


<h3>2. Определяем и рассчитываем метрики </h3>

В ходе анализа будем смотреть на 2 метрики: конверсия в покупку и ARPU.

Конверсия в покупку чувствительна, она отразит даже небольшие изменения. Также она достоверна, т.к. по полученным данным мы точно ее посчитаем. И она интерпретируема: если новая механика оплаты окажется удачной, конверсия в покупку увеличится.

Средняя выручка на пользователя также чувствительна, т.к. среднее хорошо отражает изменения. Данная метрика также достоверна, мы можем посчитать ее. И интерпретируема: если количество оплат увеличится благодаря новой механике оплаты, средняя выручка также увеличится.

In [40]:
# Рассчитаем количество активных пользователей в группах
# Рассчитаем, сколько всего пользователей в группе А и в группе B:
df_metrics = data.groupby('grp', as_index=False).id.nunique().rename(columns={'id': 'number'})
df_metrics

,grp,number
0,A,1538
1,B,6803


В контрольной группе 1538 пользователей, в тестовой - 6803. Размер тестовой группы намного больше,
необходимо уточнить, правильно ли работает система сплитования. Учтем это при проверке гипотез.

In [41]:
# Рассчитает количество пользователей, совершивших покупку
a_paid = data.query('grp == "A" and rev > 0').id.nunique()
b_paid = data.query('grp == "B" and rev > 0').id.nunique()
df_metrics['paid'] = [a_paid, b_paid]
df_metrics

,grp,number,paid
0,A,1538,78
1,B,6803,314


In [42]:
# Рассчитаем конверсию
df_metrics['CR'] = round(df_metrics.paid / df_metrics.number, 2)
df_metrics

,grp,number,paid,CR
0,A,1538,78,0.05
1,B,6803,314,0.05


In [43]:
# Рассчитаем ARPAU
# Рассчитаем суммарную выручку
a_revenue = data.query('grp == "A"').rev.sum()
b_revenue = data.query('grp == "B"').rev.sum()
df_metrics['revenue'] = [round(a_revenue, 2), round(b_revenue, 2)]
df_metrics

,grp,number,paid,CR,revenue
0,A,1538,78,0.05,72820.0
1,B,6803,314,0.05,394974.0


In [44]:
# Рассчитаем ARPAU
df_metrics['ARPU'] = round(df_metrics.revenue / df_metrics.number, 2)
df_metrics

,grp,number,paid,CR,revenue,ARPU
0,A,1538,78,0.05,72820.0,47.35
1,B,6803,314,0.05,394974.0,58.06


По предварительным расчетам, конверсии примерно равны, а вот ARPU имеют различия.

Проверим статистически, так ли это.

<h3>3. Формулируем гипотезы </h3>

Нулевая гипотеза 1: конверсии в группах А и В не имеют статистически значимых различий.
Альтернативная гипотеза 1: конверсии в группах А и В имеют статистически значимые различия.

Нулевая гипотеза 2: ARPU в группах А и В не имеют статистически значимых различий.
Альтернативная гипотеза 2: средние выручки на пользователя в группах А и В имеют статистически значимые различия.

<h3>4. Проверяем первую гипотезу (о конверсиях в покупку)</h3>

Конверсия в покупку является номинативной переменной (заплатили/не заплатили), поэтому для того, чтобы узнать есть ли статистические различия конверсий группы А и группы В, можно построить таблицу сопряженности и использовать критерий хи-квадрат.

In [45]:
# Создадим таблицу сопряженности
# Рассчитаем, сколько всего пользователей в группе А и в группе B:
a_b_number = data.groupby('grp', as_index=False).id.nunique().rename(columns={'id': 'number'})
a_b_number

,grp,number
0,A,1538
1,B,6803


In [46]:
# Рассчитаем, сколько пользователей в обеих группах совершили покупку:
a_b_rev_number = data.query('rev > 0').groupby('grp', as_index=False).id.nunique().rename(columns={'id': 'pay_number'})
a_b_rev_number

,grp,pay_number
0,A,78
1,B,314


In [47]:
# Создадим таблицу сопряженности для конверсий:
conversions_1 = pd.merge(a_b_number, a_b_rev_number, how='inner', on='grp').set_index('grp')
conversions_1 

,number,pay_number
grp,,
A,1538,78
B,6803,314


In [48]:
# Создадим столбец с теми, кто не совершил покупку, и удалим столбец с общим количеством пользователей
conversions_1['not_pay_number'] = conversions_1['number'] - conversions_1['pay_number']
conversions_1 = conversions_1.drop('number', axis=1)
conversions_1

,pay_number,not_pay_number
grp,,
A,78,1460
B,314,6489


In [49]:
# Придадим таблице более удобный вид
conversions = conversions_1.pivot_table(columns='grp')
conversions

grp,A,B
not_pay_number,1460,6489
pay_number,78,314


In [50]:
# Будем использовать критерий хи-квадрат
stats.chi2_contingency(conversions, correction=False)

(0.5821513741106591,
 0.44547028437158964,
 1,
 array([[1465.71897854, 6483.28102146],
        [  72.28102146,  319.71897854]]))

<h3>Выводы по первой гипотезе</h3>

Получили p-value равное 0.4455, что больше 0.05. Значит, мы не можем отвергнуть нулевую гипотезу о том, что конверсии
в группах А и В не имеют статистически значимых различий.
Проведенный эксперимент не позволяет нам сделать вывод, что конверсия в покупку в экспериментальной группе увеличилась.

<h3>5. Проверяем вторую гипотезу (о ARPU)</h3>

ARPU является количественной переменной. Чтобы использовать t-test, необходимо проверить, распределены ли наблюдения нормально
в обеих группах, а также равны ли дисперсии распределений.

In [51]:
# Проверим распределение в группе A на нормальность
stats.normaltest(data.query("grp=='A'").rev)

NormaltestResult(statistic=2240.595797407402, pvalue=0.0)

Получили p-value < 0.05, отклоняем нулевую гипотезу о том, что распределение является нормальным.

In [52]:
# Проверим группу B
stats.normaltest(data.query("grp=='B'").rev)

NormaltestResult(statistic=7797.399211581946, pvalue=0.0)

Распределение в группе В также не является нормальным.
Проверим равенство дисперсий с помощью теста Левена.

In [53]:
stats.levene(data.query('grp=="A"').rev, data.query('grp=="B"').rev)

LeveneResult(statistic=1.501460829433639, pvalue=0.22048133327049665)

Получили p-value = 0.22, что больше 0.05, не можем отвергнуть нулевую гипотезу о равенстве дисперсий. 
Значит, можем ипсользовать t-тест.

In [55]:
stats.ttest_ind(data.query("grp=='A'").rev, data.query("grp=='B'").rev, equal_var=True)

Ttest_indResult(statistic=-1.2253411073793445, pvalue=0.22048133326988378)

Получили p-value = 0.22.

<h3>Выводы по второй гипотезе</h3>

По гипотезе 2 мы также получили p-value > 0.05, значит, мы не можем отклонить нулевую гипотезу о том, что статистически
значимых различий в ARPU по группам нет.